# Genie Space Observability E2E Notebook

### Introduction
This notebook covers how to leverage Genie REST APIs and Databricks System Tables to create an e2e Genie Observability workflow, complete with a Genie Observability Dashboard and a Meta-Genie space to ask questions about your Genie spaces!

Running this notebook will create **Two Delta Tables**; _genie_observability_master_table_ and _genie_cost_analysis_master_table_.

- **genie_observability_master_table:** This table consists of ALL the messages sen to your genie spaces along with Genie natural langauge response and the SQL code generated as part of it. There are additional columns that provide the conversation_id, user_feedback,user_email, the statement execution ID linked to the SQL code and much more.
- **genie_cost_analysis_master_table:** This table consist of ALL information required to attribute costs to a genie space through Genie space ID. Additional columns include statement_id, user_email, start_time, cost per statement ID and much more. 

**Notes:**
- The entire solution is vibe coded (with human in the loop!), so please verify code before running in prod.
- The cost per query attribution logic is borrowed from DBSQL SME blog and dashboard. Link here: https://github.com/CodyAustinDavis/dbsql_sme/tree/main/Observability%20Dashboards%20and%20DBA%20Resources/Observability%20Lakeview%20Dashboard%20Templates/DBSQL%20Warehouse%20Advisor%20With%20Data%20Model 

**Usage Guidance:**
- You can use this notebook as part of the wider Lakehouse Adoption Dashboard and pipeline that can be deployed as part of Databricks Asset Bundles. Equally, you can download this notebook and Dashboard JSON and incorporate it into your own workflow.
- We recommend running this notebook in a test environment first and tailor the code as per your preferance.

**Pre-requisites:**
- You should be a Genie Space Author of atleast one Genie space (CAN MANAGE PERMISSION)
- You should have enough permissions to create tables in a schema within a catalog of your choice
- SELECT permission on system catalog and usage and billing schema




In [0]:
# Upgrade to the latest version of databricks_sdk package
%pip install databricks_sdk --upgrade

In [0]:
# Restart Python to ensure all libraries are reloaded
dbutils.library.restartPython()

In [0]:
# Create a text widget for catalog_name with default value 'users'
dbutils.widgets.text("catalog_name", "users")
# Create a text widget for schema_name with no default value
dbutils.widgets.text("schema_name", "")

# Retrieve the value of catalog_name from the widget
catalog_name = dbutils.widgets.get("catalog_name")
# Retrieve the value of schema_name from the widget
schema_name = dbutils.widgets.get("schema_name")
# Ensure both catalog_name and schema_name are provided
assert catalog_name and schema_name, "catalog_name and schema_name must be provided"

In [0]:
from databricks.sdk import WorkspaceClient
import pandas as pd
import json
from typing import Dict, Any, List
from datetime import datetime

# Initialize the Databricks workspace client
w = WorkspaceClient()

# Get current user information using the workspace client
current_user = w.current_user.me()
user_name = current_user.user_name

# Print the current user's username
print(f"Current user: {user_name}")

# Print the Databricks workspace host URL
print(f"Workspace: {w.config.host}")

Below we list all Genie spaces in the current workspace

In [0]:
# Initialize an empty list to store Genie space metadata
spaces = []
page_token = None

from databricks.sdk import WorkspaceClient
from datetime import datetime
import pandas as pd

# Initialize the Databricks Workspace client
w = WorkspaceClient()

# Paginate through all Genie spaces using the SDK
while True:
    response = w.genie.list_spaces(page_token=page_token)
    for s in response.spaces:
        # Append relevant space details to the list
        spaces.append({
            "space_id": getattr(s, "space_id", None),
            "name": getattr(s, "title", None),
            "description": getattr(s, "description", None),
            "warehouse_id": getattr(s, "warehouse_id", None)
        })
    # Break if there are no more pages
    if not response.next_page_token or response.next_page_token == "":
        break
    page_token = response.next_page_token

# Convert the list of spaces to a Pandas DataFrame
genie_spaces_pdf = pd.DataFrame(spaces)

In [0]:
# Display the DataFrame containing all Genie spaces metadata
display(genie_spaces_pdf)
print("---------------")
print(f"Total Genie spaces: {len(genie_spaces_pdf)}")

This code cell below is where we define the single most important function that allows us to generate the genie_observability_master_table table. You can edit this function as per your requirements or even reverse engineer this to generate newer insights!

In [0]:
import requests
import json
from typing import Dict, Any, List
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType, IntegerType
from pyspark.sql.functions import col, from_unixtime

# Cache for User Email lookups to prevent redundant API calls
USER_CACHE = {}

def get_genie_observability_table(
    space_id: str, 
    databricks_token: str, 
    host_url: str,
    include_all_users: bool = True
) -> 'pyspark.sql.DataFrame':
    """
    Fetches and constructs a Spark DataFrame containing observability data for all messages in a Genie space.

    Args:
        space_id (str): The Genie space ID.
        databricks_token (str): Databricks personal access token for authentication.
        host_url (str): Databricks workspace host URL.
        include_all_users (bool, optional): Whether to include all users' conversations. Defaults to True.

    Returns:
        pyspark.sql.DataFrame: DataFrame with observability records for the specified Genie space.
    """
    host_url = host_url.rstrip('/')
    headers = {'Authorization': f'Bearer {databricks_token}', 'Content-Type': 'application/json'}
    
    # Step 1: Get space name
    space_url = f"{host_url}/api/2.0/genie/spaces/{space_id}"
    space_resp = requests.get(space_url, headers=headers)
    space_resp.raise_for_status()
    space_name = space_resp.json().get('title', f"Space_{space_id}")
    
    # Step 2: Get conversations
    conversations = _get_all_conversations(space_id, host_url, headers, include_all_users)
    
    # Step 3: Extract and Flatten
    records = []
    for conv in conversations:
        messages = _get_all_conversation_messages(space_id, conv['conversation_id'], host_url, headers)
        for msg in messages:
            record = _extract_message_data(msg, space_id, space_name, host_url, headers)
            records.append(record)
    
    # Step 4: Create Spark DataFrame
    spark = SparkSession.builder.getOrCreate()
    schema = _get_schema()
    
    if not records:
        return spark.createDataFrame([], schema)
        
    df = spark.createDataFrame(records, schema=schema)
    
    # Convert timestamps to Datetime
    df = df.withColumn("created_datetime", from_unixtime(col("created_timestamp") / 1000).cast(TimestampType())) \
           .withColumn("last_updated_datetime", from_unixtime(col("last_updated_timestamp") / 1000).cast(TimestampType())) \
           .orderBy("created_timestamp")
    
    return df

def _get_all_conversations(space_id: str, host_url: str, headers: Dict[str, str], 
                           include_all: bool) -> List[Dict[str, Any]]:
    """
    Retrieves all conversations for a given Genie space, handling pagination.

    Args:
        space_id (str): Genie space ID.
        host_url (str): Databricks workspace host URL.
        headers (Dict[str, str]): HTTP headers for authentication.
        include_all (bool): Whether to include all users' conversations.

    Returns:
        List[Dict[str, Any]]: List of conversation metadata dictionaries.
    """
    all_conversations = []
    page_token = None
    
    while True:
        url = f"{host_url}/api/2.0/genie/spaces/{space_id}/conversations"
        params = {'page_size': 100}
        
        if page_token:
            params['page_token'] = page_token
        if include_all:
            params['include_all'] = 'true'
        
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        result = response.json()
        
        conversations = result.get('conversations', [])
        all_conversations.extend(conversations)
        
        page_token = result.get('next_page_token')
        if not page_token:
            break
    
    return all_conversations


def _get_all_conversation_messages(space_id: str, conversation_id: str, 
                                   host_url: str, headers: Dict[str, str]) -> List[Dict[str, Any]]:
    """
    Retrieves all messages from a specific conversation, handling pagination.

    Args:
        space_id (str): Genie space ID.
        conversation_id (str): Conversation ID.
        host_url (str): Databricks workspace host URL.
        headers (Dict[str, str]): HTTP headers for authentication.

    Returns:
        List[Dict[str, Any]]: List of message dictionaries.
    """
    all_messages = []
    page_token = None
    
    while True:
        url = f"{host_url}/api/2.0/genie/spaces/{space_id}/conversations/{conversation_id}/messages"
        params = {'page_size': 100}
        
        if page_token:
            params['page_token'] = page_token
        
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        result = response.json()
        
        messages = result.get('messages', [])
        all_messages.extend(messages)
        
        page_token = result.get('next_page_token')
        if not page_token:
            break
    
    return all_messages
    
def _extract_message_data(message: Dict[str, Any], space_id: str, space_name: str, host_url: str, headers: dict) -> Dict[str, Any]:
    """
    Extracts and flattens relevant fields from a Genie message object for observability.

    Args:
        message (Dict[str, Any]): Message dictionary from Genie API.
        space_id (str): Genie space ID.
        space_name (str): Genie space name.
        host_url (str): Databricks workspace host URL.
        headers (dict): HTTP headers for authentication.

    Returns:
        Dict[str, Any]: Flattened record with observability fields.
    """
    # Resolve User Email
    user_id = str(message.get('user_id')) if message.get('user_id') else None
    user_email = _resolve_email(user_id, host_url, headers)

    record = {
        'space_id': space_id,
        'space_name': space_name,
        'message_id': message.get('message_id'),
        'conversation_id': message.get('conversation_id'),
        'user_id': user_id,
        'user_email': user_email,
        'status': message.get('status'),
        'created_timestamp': message.get('created_timestamp'),
        'last_updated_timestamp': message.get('last_updated_timestamp'),
        'user_question': message.get('content'),
    }
    
    ai_responses, sql_queries, statement_ids, suggested_qs = [], [], [], []
    
    for att in message.get('attachments', []):
        # Text Attachment
        if att.get('text'):
            ai_responses.append(att['text'].get('content', ''))
        
        # Query Attachment (Corrected sibling path)
        query_obj = att.get('query')
        if query_obj:
            sql_queries.append(query_obj.get('query', ''))
            s_id = query_obj.get('statement_id')
            if s_id: statement_ids.append(str(s_id))
            
        # Suggested Questions
        if att.get('suggested_questions'):
            suggested_qs.extend(att['suggested_questions'].get('questions', []))

    # Helper to return None instead of empty string
    def join_clean(lst, sep=' | '): return sep.join(filter(None, lst)) if lst else None

    record.update({
        'ai_response': join_clean(ai_responses),
        'sql_query': join_clean(sql_queries),
        'statement_id': join_clean(statement_ids),
        'suggested_questions': join_clean(suggested_qs, sep=', '),
        'num_attachments': len(message.get('attachments', []))
    })
    
    # Feedback - Where the "Review Comments" live
    feedback = message.get('feedback', {})
    record['feedback_rating'] = feedback.get('rating', 'NONE')
        
    # Errors
    error = message.get('error', {})
    record['error_type'] = error.get('type')
    record['error_message'] = error.get('error')
    
    return record

def _resolve_email(user_id: str, host_url: str, headers: dict) -> str:
    """
    Resolves a Databricks user ID to an email address using the SCIM API, with caching.

    Args:
        user_id (str): Databricks user ID.
        host_url (str): Databricks workspace host URL.
        headers (dict): HTTP headers for authentication.

    Returns:
        str: User email if found, or a fallback string.
    """
    if not user_id or user_id in ["None", "0"]: return None
    if user_id in USER_CACHE: return USER_CACHE[user_id]
    
    try:
        url = f"{host_url}/api/2.0/preview/scim/v2/Users/{user_id}"
        resp = requests.get(url, headers=headers)
        if resp.status_code == 200:
            email = resp.json().get('userName')
            USER_CACHE[user_id] = email
            return email
    except: pass
    return f"ID_{user_id}"

def _get_schema() -> StructType:
    """
    Returns the schema for the Genie observability Spark DataFrame.

    Returns:
        StructType: Spark schema for observability records.
    """
    return StructType([
        StructField("space_id", StringType(), True),
        StructField("space_name", StringType(), True),
        StructField("message_id", StringType(), True),
        StructField("conversation_id", StringType(), True),
        StructField("user_id", StringType(), True),
        StructField("user_email", StringType(), True),
        StructField("status", StringType(), True),
        StructField("created_timestamp", LongType(), True),
        StructField("last_updated_timestamp", LongType(), True),
        StructField("user_question", StringType(), True),
        StructField("ai_response", StringType(), True),
        StructField("sql_query", StringType(), True),
        StructField("statement_id", StringType(), True),
        StructField("suggested_questions", StringType(), True),
        StructField("num_attachments", IntegerType(), True),
        StructField("feedback_rating", StringType(), True),
        StructField("error_type", StringType(), True),
        StructField("error_message", StringType(), True),
    ])


You can test the function by running the function on a Genie space

In [0]:
# Set your Genie space ID
space_id = "XXX"  # Replace with your space ID

# Use WorkspaceClient to get host and token
from databricks.sdk import WorkspaceClient

# Initialize the Databricks workspace client
w = WorkspaceClient()

# Retrieve the workspace host URL
host = w.config.host

# Retrieve the API token from the workspace client config
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Fetch Genie observability data for the specified space
df = get_genie_observability_table(space_id, token, host)

# Display the resulting DataFrame
display(df)

Below is a simple scripts that runs this function on ALL your genie spaces (currently limited to 10). 

Note: You may not have permssion to view messages of certain Genie spaces as you may not have access to the underlying tables. In this case you will see a lot of NULLs

In [0]:
from databricks.sdk import WorkspaceClient
from datetime import datetime
import pandas as pd

# Initialize the Databricks workspace client
w = WorkspaceClient()

# Get authentication parameters
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
host = w.config.host

# Fetch all Genie spaces
print("🔍 Fetching all Genie spaces...")
spaces = []
page_token = None

while True:
    response = w.genie.list_spaces(page_token=page_token)
    for s in response.spaces:
        spaces.append({
            "space_id": getattr(s, "space_id", None),
            "name": getattr(s, "title", None),
            "description": getattr(s, "description", None),
            "warehouse_id": getattr(s, "warehouse_id", None)
        })
    if not response.next_page_token or response.next_page_token == "":
        break
    page_token = response.next_page_token

print(f"✓ Found {len(spaces)} Genie spaces")

# Limit to 10 spaces
MAX_SPACES = 10
if len(spaces) > MAX_SPACES:
    print(f"⚠ Limiting processing to first {MAX_SPACES} spaces\n")
    spaces = spaces[:MAX_SPACES]
else:
    print()

# Collect observability data from all spaces
all_dfs = []

for i, space in enumerate(spaces, 1):
    space_id = space['space_id']
    space_name = space['name']
    
    print(f"\n{'='*80}")
    print(f"[{i}/{len(spaces)}] Processing Space: {space_name}")
    print(f"Space ID: {space_id}")
    print(f"{'='*80}")
    
    try:
        # Get observability data for this space
        df_space = get_genie_observability_table(space_id, token, host)
        
        if df_space.count() > 0:
            all_dfs.append(df_space)
            print(f"\n✓ Successfully extracted {df_space.count()} messages from {space_name}")
        else:
            print(f"\n⚠ No messages found in {space_name}")
            
    except Exception as e:
        print(f"\n❌ Error processing space {space_name}: {str(e)}")
        continue

# Combine all DataFrames
if all_dfs:
    print(f"\n\n{'='*80}")
    print("📊 COMBINING ALL RESULTS")
    print(f"{'='*80}")
    
    # Union all DataFrames
    df = all_dfs[0]
    for df_next in all_dfs[1:]:
        df = df.union(df_next)
    
    total_messages = df.count()
    total_spaces = df.select("space_id").distinct().count()
    
    print(f"\n✅ SUCCESS!")
    print(f"   Total Spaces Processed: {total_spaces}")
    print(f"   Total Messages Extracted: {total_messages}")
    print(f"\n{'='*80}\n")
    
    display(df)
else:
    print("\n⚠ No data found across any Genie spaces")
    df = None

Save the table as a delta table and add a nice detailed table description!

In [0]:
# Write the observability data to a Delta table
df.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.genie_observability_master_table")

# Add a detailed table description
spark.sql(f"""
  COMMENT ON TABLE {catalog_name}.{schema_name}.genie_observability_master_table IS 
  'Comprehensive observability table for Genie AI/BI spaces. Contains detailed message-level data including user questions, AI responses, generated SQL queries, execution metadata, user feedback ratings, and error information. Used for monitoring Genie usage, analyzing query patterns, and tracking user engagement across all accessible Genie spaces.'
""")

We successfully created the **genie_observability_master_table** table! Now lets proceed to create the **genie_cost_analysis_master_table** table

**An Aside: How we approached attributing costs from a Genie Space.**

The Strategy: We used a "Time-Weighted" approach. It treats the total cost of a warehouse as a "pie" and slices it up based on how long each query took to run relative to other queries.

Step 1: Calculate the Total Bill (Per Warehouse) First, we look at the system billing logs (system.billing.usage) to calculate the total price tag (in DBUs and Dollars) for every SQL warehouse that was active during our specified dates.

Step 2: Measure "Work" (Per Query) we look at the query history (system.query.history) and defines "work" as Duration (specifically, Compilation Time + Execution Time).

We calculate how long a specific query took.

We then calculate the Total Duration of all queries that ran on that same warehouse during that time window (using a window function).

Step 3: Calculate the Ratio (The Slice) It compares the two numbers from Step 2 to create a percentage:

"This query took 1 minute. The warehouse ran queries for a total of 100 minutes. Therefore, this query is responsible for 1% of the bill."

Step 4: Assign the Cost It multiplies that percentage (1%) by the total warehouse bill found in Step 1 to assign a specific dollar cost to the query.

Step 5: Filter for Genie Finally, it filters the results to show only the queries that originated from your specific Genie Space (genie_space_id), letting you see exactly how much that specific AI agent cost you.

In [0]:
# %sql
# -- Use this query to attribute compute costs to a single Genie space. Copy and run in the SQL editor to test for your space. We will not be running this as part of this notebook
# WITH warehouse_usage AS (
#   -- Capture SQL compute costs for each warehouse within the specified date range.
#   SELECT
#     u.usage_metadata.warehouse_id,
#     SUM(u.usage_quantity) AS total_billed_dbus, -- Total DBUs billed for the warehouse
#     SUM(u.usage_quantity * p.pricing.default) AS total_billed_dollars -- Total cost in dollars for the warehouse
#   FROM system.billing.usage u
#   JOIN system.billing.list_prices p 
#     ON u.sku_name = p.sku_name
#     AND u.usage_start_time >= p.price_start_time
#     AND (u.usage_start_time < p.price_end_time OR p.price_end_time IS NULL)
#   WHERE u.usage_start_time >= :start_date::timestamp -- Filter by start date
#     AND u.usage_end_time <= :end_date::timestamp     -- Filter by end date
#     AND u.usage_unit = 'DBU'                         -- Only DBU usage
#     AND u.sku_name ILIKE '%SQL%'                     -- Only SQL compute SKUs
#   GROUP BY 1
# ),
# query_base AS (
#   -- Gather query execution details and compute total execution time per query.
#   SELECT
#     statement_id, -- Unique query statement ID
#     executed_by,  -- User who executed the query
#     statement_text, -- The SQL code executed
#     compute.warehouse_id, -- Warehouse used for the query
#     query_source.genie_space_id, -- Genie space ID associated with the query
#     start_time, -- Query start time
#     -- Total time spent compiling and executing the query
#     (
#       COALESCE(compilation_duration_ms, 0) + 
#       COALESCE(execution_duration_ms, 0)
#     ) AS total_accurate_duration_ms,
#     -- Total warehouse activity time for proportional allocation
#     SUM(
#       COALESCE(compilation_duration_ms, 0) + 
#       COALESCE(execution_duration_ms, 0)
#     ) OVER (PARTITION BY compute.warehouse_id) AS total_warehouse_activity_ms
#   FROM system.query.history
#   WHERE start_time >= :start_date::timestamp -- Filter by start date
#     AND start_time <= :end_date::timestamp   -- Filter by end date
#     AND compute.warehouse_id IS NOT NULL     -- Only queries with a warehouse
# ),
# allocated_metrics AS (
#   -- Allocate DBU and dollar costs to each query based on its share of warehouse activity.
#   SELECT
#     q.statement_id,
#     q.executed_by,
#     q.statement_text,
#     q.genie_space_id,
#     q.start_time,
#     q.warehouse_id,
#     q.total_accurate_duration_ms,
#     -- Proportion of warehouse activity attributed to this query
#     (q.total_accurate_duration_ms / NULLIF(q.total_warehouse_activity_ms, 0)) AS work_proportion,
#     u.total_billed_dbus,    -- Total DBUs billed for the warehouse
#     u.total_billed_dollars  -- Total dollars billed for the warehouse
#   FROM query_base q
#   -- Use LEFT JOIN to ensure queries are included even if billing data is missing
#   LEFT JOIN warehouse_usage u ON q.warehouse_id = u.warehouse_id
# )
# SELECT 
#   statement_id, -- Query statement ID
#   executed_by AS user_email, -- User email who ran the query
#   start_time, -- Query start time
#   genie_space_id, -- Genie space ID
#   warehouse_id, -- Warehouse ID
#   ROUND(total_accurate_duration_ms / 1000, 2) AS accurate_duration_seconds, -- Query duration in seconds
#   ROUND(work_proportion * total_billed_dbus, 4) AS dbus_consumed,           -- DBUs consumed by the query
#   ROUND(work_proportion * total_billed_dollars, 4) AS cost_usd,             -- Cost in USD for the query
#   statement_text AS sql_code -- The SQL code executed
# FROM allocated_metrics
# WHERE genie_space_id = :genie_space_id -- Filter for the specified Genie space
# ORDER BY start_time DESC; -- Show most recent queries first

In the code below we use system tables and the ability to link query execution source to Genie Space ID to calculate the costs that arise from a genie space

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.{schema_name}.genie_cost_analysis_master_table AS
WITH date_range AS (
  -- Determine the analysis window: from the earliest Genie message to today.
  SELECT 
    MIN(created_datetime) AS start_date,
    CURRENT_DATE() AS end_date
  FROM {catalog_name}.{schema_name}.genie_observability_master_table
),
warehouse_usage AS (
  -- Aggregate total DBU and dollar costs for each SQL warehouse in the date range.
  SELECT
    u.usage_metadata.warehouse_id,
    SUM(u.usage_quantity) AS total_billed_dbus, -- Total DBUs billed for the warehouse
    SUM(u.usage_quantity * p.pricing.default) AS total_billed_dollars -- Total cost in dollars
  FROM system.billing.usage u
  JOIN system.billing.list_prices p 
    ON u.sku_name = p.sku_name
    AND u.usage_start_time >= p.price_start_time
    AND (u.usage_start_time < p.price_end_time OR p.price_end_time IS NULL)
  CROSS JOIN date_range dr
  WHERE u.usage_start_time >= dr.start_date -- Only usage within the Genie message window
    AND u.usage_end_time <= dr.end_date
    AND u.usage_unit = 'DBU'                -- Only DBU usage (compute)
    AND u.sku_name ILIKE '%SQL%'            -- Only SQL compute SKUs
  GROUP BY 1
),
query_base AS (
  -- Gather all Genie-related queries and compute their execution durations.
  SELECT
    statement_id, -- Unique query statement ID
    executed_by,
    statement_text,
    compute.warehouse_id,
    query_source.genie_space_id,
    start_time,
    (COALESCE(compilation_duration_ms, 0) + COALESCE(execution_duration_ms, 0)) AS total_accurate_duration_ms,
    SUM(COALESCE(compilation_duration_ms, 0) + COALESCE(execution_duration_ms, 0)) OVER (PARTITION BY compute.warehouse_id) AS total_warehouse_activity_ms
  FROM system.query.history
  CROSS JOIN date_range dr
  WHERE start_time >= dr.start_date -- Only queries within the Genie message window
    AND start_time <= dr.end_date
    AND compute.warehouse_id IS NOT NULL
),
allocated_metrics AS (
  -- Allocate warehouse costs to each query based on its share of total warehouse activity.
  SELECT
    q.statement_id,
    q.executed_by,
    q.statement_text,
    q.genie_space_id,
    q.start_time,
    q.warehouse_id,
    q.total_accurate_duration_ms,
    (q.total_accurate_duration_ms / NULLIF(q.total_warehouse_activity_ms, 0)) AS work_proportion,
    u.total_billed_dbus,
    u.total_billed_dollars
  FROM query_base q
  LEFT JOIN warehouse_usage u ON q.warehouse_id = u.warehouse_id -- Include queries even if billing data is missing
)
SELECT 
  statement_id,
  executed_by AS user_email,
  start_time,
  genie_space_id,
  warehouse_id,
  ROUND(total_accurate_duration_ms / 1000, 2) AS accurate_duration_seconds,
  ROUND(work_proportion * total_billed_dbus, 4) AS dbus_consumed,
  ROUND(work_proportion * total_billed_dollars, 4) AS cost_usd,
  statement_text AS sql_code
FROM allocated_metrics
WHERE genie_space_id is NOT NULL
ORDER BY start_time DESC
""")
print("Table created successfully.")

In [0]:
# Add a detailed table description
spark.sql(f"""
  COMMENT ON TABLE {catalog_name}.{schema_name}.genie_cost_analysis_master_table IS 
  'This table provides a granular cost attribution analysis for Databricks Genie spaces. It links individual SQL queries executed within Genie to the underlying SQL Warehouse compute costs. By calculating the "work proportion" of every query relative to the warehouse''s total activity, it estimates the specific DBU consumption and USD cost for each query statement.'
""")

This concludes this notebook. We now have created two critical tables that will help us monitor and analyse our genie spaces across multiple dimensions:
- Cost attribution and chargeback
- Usability and accuracy
- Insights to further improve the space by adding more tables, adding SQL examples, trusted assets etc



### Take this to the next level
You can enhance this notebook by incorporating more advanced techniques such as:
- Run topic modelling on user user questions to identify common patterns that can help you improve youe Genie space
- Incorporate more advanced cost attribution strategies for charge backs. See [LINK](https://github.com/databrickslabs/sandbox/tree/main/dbsql/cost_per_query/PrPr) and [LINK](https://www.databricks.com/resources/demos/tutorials/governance/system-tables?itm_data=demo_center) for inspiration.
- Incorporate this notebook as part of your custome Databricks Jobs process